In [71]:
import pandas as pd

data = pd.read_csv('data/dat.csv', sep=';')



In [72]:
data['Образование'] = data['Образование и ВУЗ'].apply(lambda x: ' '.join(x.split(' ')[:2]))

data = data.drop(['Образование и ВУЗ'], axis=1)

mided = data[data['Образование'] == 'Среднее образование']
mided.shape[0]


559

In [73]:
data['Пол'] = data['Пол, возраст'].apply(lambda x: x[:1])
data['Возраст'] = data['Пол, возраст'].apply(lambda x: x.split(' ')[3])

midage = round(data['Возраст'].astype(int).mean())
print(midage)
countw = data[data['Пол'] == 'Ж'].shape[0]
data = data.drop(['Пол, возраст'], axis=1)                                 
wom = round(countw / data.shape[0] * 100, 2)                                             
print(wom)


32
19.07


In [74]:
import numpy as np

def trans(arg):
    if arg is np.nan:
        return np.nan
    if arg == 'Не указано':
        return np.nan
    month_id = ['месяцев', 'месяца', 'месяц']
    year_id = ['лет', 'год', 'года']
    splitted = str(arg).split(' ')[2:]
    if splitted[1] in year_id:
        mon = int(splitted[0]) * 12
        if splitted[3] in month_id:
            totmon = int(splitted[2]) + mon
            return totmon
        else: return mon
    if splitted[1] in month_id:
        mon = int(splitted[0])
        return mon
    else: pass
data['Опыт работы (месяц)'] = data['Опыт работы'].apply(trans)

print(data['Опыт работы (месяц)'].median())




100.0


In [75]:

def cityloc(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань',
                      'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск',
                      'Пермь', 'Воронеж', 'Волгоград' ]
    splitted = str(arg).split(' ')

    if splitted[0] == 'Санкт-Петербург' or splitted[0] == 'Москва':
        return splitted[0]
    elif splitted[0] in million_cities:
        return 'город-милионник'
    else:
        return 'Другие'

    
    
def citymove(arg):
    splitted = str(arg).split(',')
    dbsplit = splitted[-2].split(' ')
    if 'не' in dbsplit:
        return False
    else:
        return True
 

def comand(arg):
    splitted = str(arg).split(',')
    dbsplit = splitted[-1].split(' ')
    if 'не' in dbsplit:
        return False
    else:
        return True
  
data['Город'] = data['Город, переезд, командировки'].apply(cityloc)
spb = data[data['Город'] == 'Санкт-Петербург'].shape[0]
spb_proc = round(spb / data.shape[0] * 100)
print(spb_proc)


data['Готовность к переезду'] = data['Город, переезд, командировки'].apply(citymove)
data['Готовность к командировкам'] = data['Город, переезд, командировки'].apply(comand)


mask = (data['Готовность к переезду'] == True) & (data['Готовность к командировкам'] == True)

movecom = data[mask].shape[0]
proc = round(movecom / data.shape[0] * 100)
print(proc)

data = data.drop(['Город, переезд, командировки'], axis=1)

11
32


In [76]:
busyTypes = ['полная занятость', 'частичная занятость', 'проектная работа', 'стажировка', 'волонтерство']
data['полная занятость'] = data['Занятость'].apply(lambda x: True if ('полная занятость' in x) else False)

data['частичная занятость'] = data['Занятость'].apply(lambda x: True if ('частичная занятость' in x) else False)

data['проектная работа'] = data['Занятость'].apply(lambda x: True if ('проектная работа' in x) else False)

data['стажировка'] = data['Занятость'].apply(lambda x: True if ('стажировка' in x) else False)

data['волонтерство'] = data['Занятость'].apply(lambda x: True if ('волонтерство' in x) else False)

msk = (data['проектная работа'] == True) & (data['волонтерство'] == True)
print(data[msk].shape[0])


data['гибкий график'] = data['График'].apply(lambda x: True if ('гибкий график' in x) else False)

data['полный день'] = data['График'].apply(lambda x: True if ('полный день' in x) else False)

data['сменный график'] = data['График'].apply(lambda x: True if ('сменный график' in x) else False)

data['удаленная работа'] = data['График'].apply(lambda x: True if ('удаленная работа' in x) else False)

data['вахтовый метод'] = data['График'].apply(lambda x: True if ('вахтовый метод' in x) else False)

msk2 = (data['вахтовый метод'] == True) & (data['гибкий график'] == True)
print(data[msk2].shape[0])



436
2311


In [77]:
exrates = pd.read_csv('data/ExchangeRates.csv')

def get_salary_num(arg):
    salary = float(arg.split(' ')[0])
    return salary

def get_salary_currency(arg):
    currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': 'BYN',
        'EUR': 'EUR', 'KGS': 'KGS',
        'сум': 'UZS', 'AZN': 'AZN'
    }
    curr = arg.split(' ')[1].replace('.', '')
    if curr == 'руб':
        return 'RUB'
    else:
        return currency_dict[curr]
    
    
exrates['date'] = pd.to_datetime(exrates['date']).dt.date
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме']).dt.date
data['ЗП (tmp)'] = data['ЗП'].apply(get_salary_num)
data['Курс (tmp)'] = data['ЗП'].apply(get_salary_currency)


merged = data.merge(exrates, left_on=['Курс (tmp)', 'Обновление резюме'], right_on=['currency', 'date'], how='left')

merged['close'] = merged['close'].fillna(1)
merged['proportion'] = merged['proportion'].fillna(1)

data['ЗП (руб)'] = merged['close'] * merged['ЗП (tmp)'] / merged['proportion']
data = data.drop(['ЗП', 'ЗП (tmp)', 'Курс (tmp)'], axis=1)

print(round(data['ЗП (руб)'].median()/1000))


59


In [79]:
data.info()
data.to_csv('data/nd2.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Ищет работу на должность:        44744 non-null  object 
 1   Занятость                        44744 non-null  object 
 2   График                           44744 non-null  object 
 3   Опыт работы                      44576 non-null  object 
 4   Последнее/нынешнее место работы  44743 non-null  object 
 5   Последняя/нынешняя должность     44742 non-null  object 
 6   Обновление резюме                44744 non-null  object 
 7   Авто                             44744 non-null  object 
 8   Образование                      44744 non-null  object 
 9   Пол                              44744 non-null  object 
 10  Возраст                          44744 non-null  object 
 11  Опыт работы (месяц)              44574 non-null  float64
 12  Город             